In [21]:
!pip install catboost imbalanced-learn pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import xgboost as xgb


In [4]:
email_data = pd.read_csv('/content/preprocessed_data.csv')
email_data

,Unnamed: 0,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,opened,clicked,purchase_bins,purchase_bins_encoded,weekday_encoded,email_text_encoded,email_version_encoded,user_country_encoded
0,0,85120,short_email,personalized,2,Sunday,US,5,0,0,0-5,0,3,0,1,0
1,1,966622,long_email,personalized,12,Sunday,UK,2,1,1,0-5,0,3,1,1,1
2,2,777221,long_email,personalized,11,Wednesday,US,2,0,0,0-5,0,6,1,1,0
3,3,493711,short_email,generic,6,Monday,UK,1,0,0,0-5,0,1,0,0,1
4,4,106887,long_email,generic,14,Monday,US,6,0,0,6-10,3,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,803504,short_email,personalized,4,Monday,US,5,0,0,0-5,0,1,0,1,0
99996,99996,899722,long_email,personalized,7,Saturday,US,1,0,0,0-5,0,2,1,1,0
99997,99997,449610,long_email,personalized,11,Saturday,UK,6,0,0,6-10,3,2,1,1,1
99998,99998,72497,short_email,generic,10,Monday,UK,0,0,0,0-5,0,1,0,0,1


In [37]:
features = ['purchase_bins_encoded', 'weekday_encoded',
       'email_text_encoded', 'email_version_encoded', 'user_country_encoded','hour']

X = email_data[features]
y = email_data[['opened','clicked']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [38]:
# from imblearn.over_sampling import ADASYN

# adasyn = ADASYN(random_state=42)
# X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

# print(f"Original class distribution: {y_train.value_counts().to_dict()}")
# print(f"Resampled class distribution: {np.bincount(y_resampled)}")

In [39]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score, roc_auc_score


In [62]:
cat_features = features

# Initialize the CatBoost model
model = CatBoostClassifier(
    iterations=5000,
    learning_rate=0.01,
    depth=6,
    l2_leaf_reg=3.0,
    random_strength=1.5,
    bagging_temperature=1.0,
    early_stopping_rounds=50,
    class_weights=[1, 8],

    use_best_model=True,
    cat_features=cat_features,
    random_seed=42,
    verbose=50
)




In [63]:
model.fit(X_train, y_train['opened'], eval_set=(X_test, y_test['opened']))

0:	learn: 0.6923791	test: 0.6923748	best: 0.6923748 (0)	total: 118ms	remaining: 9m 47s
50:	learn: 0.6710982	test: 0.6711110	best: 0.6711110 (50)	total: 3.3s	remaining: 5m 20s
100:	learn: 0.6611575	test: 0.6612088	best: 0.6612088 (100)	total: 6.77s	remaining: 5m 28s
150:	learn: 0.6562235	test: 0.6564346	best: 0.6564346 (150)	total: 11.6s	remaining: 6m 13s
200:	learn: 0.6533825	test: 0.6537008	best: 0.6537008 (200)	total: 15s	remaining: 5m 57s
250:	learn: 0.6518755	test: 0.6522893	best: 0.6522893 (250)	total: 17.9s	remaining: 5m 38s
300:	learn: 0.6506883	test: 0.6512697	best: 0.6512697 (300)	total: 20.9s	remaining: 5m 26s
350:	learn: 0.6498217	test: 0.6505853	best: 0.6505853 (350)	total: 25.4s	remaining: 5m 36s
400:	learn: 0.6491832	test: 0.6501086	best: 0.6501086 (400)	total: 28.2s	remaining: 5m 23s
450:	learn: 0.6487063	test: 0.6497641	best: 0.6497641 (450)	total: 31.1s	remaining: 5m 13s
500:	learn: 0.6482703	test: 0.6494813	best: 0.6494813 (499)	total: 34.1s	remaining: 5m 6s
550:	lear

In [64]:
y_pred_catb = model.predict(X_test)
f1 = f1_score(y_test['opened'], y_pred_catb)
print("F1 Score on test set:", f1)


F1 Score on test set: 0.25362602900823206


In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test['opened'], y_pred_catb))


              precision    recall  f1-score   support

           0       0.93      0.62      0.74     17931
           1       0.16      0.63      0.25      2069

    accuracy                           0.62     20000
   macro avg       0.55      0.62      0.50     20000
weighted avg       0.85      0.62      0.69     20000



In [45]:
dtrain = xgb.DMatrix(X_train, label=y_train['opened'])
dtest = xgb.DMatrix(X_test, label=y_test['opened'])

In [46]:
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 7,
    'eta': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 1.0,
    'alpha': 0.5,
    'seed': 42,
    'scale_pos_weight': 8,

}


In [47]:
watchlist = [(dtrain, 'train'), (dtest, 'test')]

# Train with early stopping
bst = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=2000,
    evals=watchlist,
    early_stopping_rounds=50,
    verbose_eval=50

)

[0]	train-logloss:0.65918	test-logloss:0.65906
[50]	train-logloss:0.61701	test-logloss:0.61495
[100]	train-logloss:0.61068	test-logloss:0.61029
[150]	train-logloss:0.60761	test-logloss:0.60888
[200]	train-logloss:0.60468	test-logloss:0.60753
[250]	train-logloss:0.60251	test-logloss:0.60677
[300]	train-logloss:0.60153	test-logloss:0.60725
[322]	train-logloss:0.60117	test-logloss:0.60750


In [49]:
y_pred_prob = bst.predict(dtest, iteration_range=(0, bst.best_iteration + 1))
y_pred_xgb = (y_pred_prob > 0.5).astype(int)

# Evaluate

f1_score = f1_score(y_test['opened'], y_pred_xgb)
print(f"F1_score: {f1_score:.4f}")
print(f"Best iteration: {bst.best_iteration}")
print(f"Best score: {bst.best_score}")

F1_score: 0.2465
Best iteration: 272
Best score: 0.6065594956377288


In [50]:
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
X_train_np = X_train.values
X_test_np = X_test.values
y_train_np = y_train['opened'].values.flatten()
y_test_np = y_test['opened'].values.flatten()

# Step 2: Initialize and train TabNet
tabnet_model = TabNetClassifier(
    n_d=16, n_a=16, n_steps=5,
    gamma=1.5, lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax',
    verbose=True,
    seed=42
)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [51]:

tabnet_model.fit(
    X_train_np, y_train_np,
    eval_set=[(X_test_np, y_test_np)],
    eval_name=["val"],
    eval_metric=["logloss"],
    max_epochs=20,
    patience=5,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    weights={0:1,1:8}

)

epoch 0  | loss: 0.73223 | val_logloss: 0.71575 |  0:00:06s
epoch 1  | loss: 0.66164 | val_logloss: 0.65587 |  0:00:14s
epoch 2  | loss: 0.65618 | val_logloss: 0.67765 |  0:00:21s
epoch 3  | loss: 0.65633 | val_logloss: 0.66407 |  0:00:29s
epoch 4  | loss: 0.65451 | val_logloss: 0.63967 |  0:00:37s
epoch 5  | loss: 0.65639 | val_logloss: 0.63666 |  0:00:44s
epoch 6  | loss: 0.65521 | val_logloss: 0.61197 |  0:00:52s
epoch 7  | loss: 0.65357 | val_logloss: 0.62381 |  0:00:58s
epoch 8  | loss: 0.65347 | val_logloss: 0.63085 |  0:01:06s
epoch 9  | loss: 0.6558  | val_logloss: 0.61795 |  0:01:14s
epoch 10 | loss: 0.65546 | val_logloss: 0.65252 |  0:01:21s
epoch 11 | loss: 0.65467 | val_logloss: 0.63841 |  0:01:29s

Early stopping occurred at epoch 11 with best_epoch = 6 and best_val_logloss = 0.61197


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [53]:
from sklearn.metrics import f1_score
# Predictions
preds_proba = tabnet_model.predict_proba(X_test_np)[:, 1]
preds_tabnet = (preds_proba > 0.5).astype(int)

# Metrics
accuracy = accuracy_score(y_test_np, preds_tabnet)
precision = precision_score(y_test_np, preds_tabnet)
recall = recall_score(y_test_np, preds_tabnet)
f1 = f1_score(y_test_np, preds_tabnet)


# Output results
print("\nTabNet Performance:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")




TabNet Performance:
Accuracy:  0.6268
Precision: 0.1565
Recall:    0.5940
F1 Score:  0.2477


In [54]:
def majority_vote(preds1, preds2,preds3):
    preds = np.array([preds1, preds2,preds3])
    # Sum predictions and take majority vote
    maj_vote = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=preds)
    return maj_vote

In [55]:
ensemble_preds = majority_vote(y_pred_catb, preds_tabnet, y_pred_xgb)

In [66]:

# Evaluate accuracy
accuracy = accuracy_score(y_test['opened'], ensemble_preds)
precision = precision_score(y_test['opened'], ensemble_preds)
recall = recall_score(y_test['opened'], ensemble_preds)
f1 = f1_score(y_test['opened'], ensemble_preds)

print(f"Ensemble Accuracy: {accuracy:.4f}")
print(f"Ensemble Precision: {precision:.4f}")
print(f"Ensemble Recall: {recall:.4f}")
print(f"Ensemble F1 Score: {f1:.4f}")

# Compare with individual models
print("\nIndividual Model Performance:")
print(f"CatBoost F1: {f1_score(y_test['opened'], y_pred_catb):.4f}")
print(f"XGBoost F1: {f1_score(y_test['opened'], y_pred_xgb):.4f}")
print(f"Tabnet F1 : {f1_score(y_test['opened'],preds_tabnet)}")

Ensemble Accuracy: 0.6449
Ensemble Precision: 0.1600
Ensemble Recall: 0.5727
Ensemble F1 Score: 0.2502

Individual Model Performance:
CatBoost F1: 0.2536
XGBoost F1: 0.2465
Tabnet F1 : 0.24773231203386414


In [42]:
X_train.to_csv("X_train.csv")
y_train.to_csv("y_train.csv")
X_test.to_csv("X_test.csv")
y_test.to_csv("y_test.csv")

In [44]:
pd.read_csv('/content/y_train.csv')

,Unnamed: 0,opened,clicked
0,15067,0,0
1,12578,0,0
2,42594,0,0
3,44105,0,0
4,27801,0,0
...,...,...,...
79995,60032,1,0
79996,60965,0,0
79997,96492,0,0
79998,14049,0,0
